In [1]:
#!/usr/bin/env python
# coding: utf-8
#

# In[1]:
import sys
sys.path.insert(-1, "c:\python\lib\site-packages\lmfit")
import re
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from lmfit import Model
from numpy import exp, loadtxt, sqrt, std, mean
from scipy import stats
from scipy.stats import sem
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', None)
import math
import statistics
from scipy.stats import pearsonr

FolderPathUsers = 'Fiammetta/Desktop/tirocinio' #OR, 'mauromanassi/Dropbox/MANASSILAB/zzz_FIAMMETTA' 



# In[2]:

def RunAnalysis(path):

### Parameters

    #### User's Input
    halfway = 74 # circular space correction (74 morphs, 90 orientation)
    step = 20 # for running mean, how wide to run avg.
    nBack = 1 #how many trials to go back/forward if+ 1Back if- 1Forward
    Forward = 0 #0 not forward, 1 is forward
    Timecutoff = 10 # if exceeds this RT, then filtered
    ErrorCutoff = 40 # for error (bar - gabor) filter
    polynum = 15 # polynomial correction, highest exponent


    #### Control console
    TimelimitYN = 1 # remove over-RT?
    SDcutoffYN = 0 # cutoff as 2 s.d.
    FilterYN = 1 # Filter data for Error?
    polyYN = 0 # do polynomial correction?
    DoGYN = 1 # plot DoG curve?
    bootstrapYN = 0 # do bootstrap sampling?
    permYN = 0 # do permutation sampling?



### Looks For Files In Folder

    ##### read all csv files
    files = os.listdir(path) # list of ALL files
    print(files)
    files_csv = list(filter(lambda x: x[-4:] == '.csv' , files)) # list of all CSV files
    print(files_csv)
    FileNameList = []
    for file in files_csv:
        FileName = file[-7:-4]
        FileNameList.append(FileName) #to give random name to file saved NOT USED

    ##### if result exists, don't run function again
    ResultPath = path + 'results/'

    ##### show CSV files to analyze
    print('FILES FOUND AND READ:',files_csv)

### Data Assembling

    #MAIN VARIABLES
    ActualStim = 'stimulusID'
    #StartResp = 'StartBar'
    Response = 'morphID'
    trialsLoop = 'trialNumber' #trialnum per block trialsLoop.thisRepN
    blocksLoop = 'blockNumber' #blocknum blocksLoop.thisRepN
    #GaborTime = 'GaborTime'
    #MaskTime = 'MaskTime'
    #ISItime = 'ISI' #blocknum blocksLoop.thisRepN
    #TimeResp = 'RespTime'
    #ITI = 'ITI'
    Location = 'stimLocationDeg'
    #Gender = '*Geschlecht (M/W/anders)'
    #age = '*Geburtsjahr '
    #PersonalCode = 'Bitte hier eingeben: (1) die ersten 2 Buchstaben des Vornamens Ihrer Mutter, (2) der Tag (des Monats) an dem Sie geboren wurden, (3) die letzten 2 Buchstaben Ihres Vornamens, (4) der Tag (des Monats) an dem Ihre Mutter geboren wurde (Bsp: LY16KE26)'
    blockType_ = 'blockType'
    error_corr = 'errorCorrected'

    


    # define a void list to store data
    data_list = pd.DataFrame() #dataframe

    # transformation
    #num_filter = re.compile(r'\d+')

    #read useful columns, save in all_data
    for file in  files_csv:
        tmp = (pd.read_csv(path + file)[[ActualStim, Response, error_corr, trialsLoop, blocksLoop,Location,blockType_]])
        all_data = pd.concat([data_list, tmp],ignore_index=True) #if all together: ignore_index=True
        
    #print(all_data)

    
    # computes trial number per block, block number and total trial number
    file_csv = len(files_csv) #only for online studies, total of files analyzed
    trials = (np.nanmax(all_data[trialsLoop])) #+1 because it starts at 0
    blocks = (np.nanmax(all_data[blocksLoop])) #+1 because it starts at 0

    PerFile = (trials*blocks) # Trials in each csv file
    BlocksTotal = blocks*file_csv # Blocks in all csv file
    TotalTrial = trials * blocks * file_csv # SuperTotal Trials

    print(file_csv,'file_csv(s):\n', int(trials),'trials/blocks.', int(blocks),'blocks - each file_csv.\n', int(PerFile) ,'trials - each file_csv.\n', int(BlocksTotal),'total blocks TOTAL.', int(TotalTrial),'total trials TOTAL.')

    # delete invalid rows and useless columns
    all_data.dropna(axis = 0, how = 'any', inplace = True) # exclude NaN (often between blocks)
    all_data = all_data[all_data['blockType']=='Experiment']
    #print(all_data)
    
    
    

### Data Assembling

    #plot_dataTOTAL = all_data

    xdata = all_data['stimulusID']
    CurrStim = np.array(xdata)
    Respdata = all_data['morphID']
    ydata = all_data[error_corr] #np.array(Respdata)-np.array(xdata) # ERROR
    
            
            
    # Filter for outliers in error(ydata)
   # if SDcutoffYN:
   #     ErrorCutoff = 2 * std(ydata) # otherwise ErrorCutoff at the beginning
   # if FilterYN == 1:
   #     xdata = xdata[abs(ydata) < ErrorCutoff]
   #     CurrStim = CurrStim[abs(ydata) < ErrorCutoff]
   #     #Blocks_ = Blocks_[abs(ydata) < ErrorCutoff]
   #     ydata = ydata[abs(ydata) < ErrorCutoff]
   #     std_ydata=np.std(ydata)
   #     std_ydata=round(std_ydata,2)
   #     #print("Resp Err NO FILTER =",std_ydataNOFILTER,"Response Error After Outlier Removal =",std_ydata,", cutoff threshold:", int(ErrorCutoff), ",", "trials removed", int(ydataAfterConditions - len(ydata)))
   # else:
   #     std_ydata=np.std(ydata)
   #     std_ydata=round(std_ydata,2)
   #     #print("Response Error No Filter =",std_ydata)
   # 
    
    #compensates for biases
    #ydata = ydata-np.mean(ydata)---> DA FARE??  
    
    
    #DEFINING GRAPH VARIABLES
    ydata_neg = ydata #error, with negative values   #rel_error e #abs_error
    ydata = abs(ydata) #absolute error

        
    #### CREATE A DATAFRAME
    df = pd.DataFrame({"ActualZ": xdata, "Error": ydata, "Error_neg": ydata_neg})
    pd.set_option("max_rows", None)
    df = df.sort_values(by=["ActualZ"])
    #print(df)

    #Bin the errors in 49 bins (3 current orientations in each bin)
    #slices = np.linspace(1, 145, (49), endpoint=False).astype(int)
    #print(slices)
    #print(len(slices))

    slices2 = []
    for x in range(-2,145,3):
        x2 = x+3
        #print(x2)
        slices2.append(x2)
    #print(slices2)
    #print(len(slices2))
    
    TotBins= []
    for x in slices2:
        #print(x)
        bins_= df[(df["ActualZ"]>=x) & (df["ActualZ"]<x+3)]
        #print(bins_)
        bins_2 = list(bins_["Error_neg"])
        #print(bins_2)
        TotBins.append(bins_2)                  
    
    
    TotBins_allsubjects.append(TotBins)
        
    print(TotBins_allsubjects)


    
    
    

# ### Run Function ###
# In[3]:


#############################################Runs Function#############################################
pathSTART = "/Users/" + FolderPathUsers + "/SuperRecognizers/TryConsistencyAnalysis/Data/"


All_ErrorOut = []
TotAge = []
TotAmplitude = []
TotAmplitude2 = []
Totmean = []
Totmean_abs = []
SEM = []
Totmean1 = []

TotBins_allsubjects = []



print(pathSTART)
Subj = sorted(os.listdir(pathSTART))
print(Subj)
TotalSubj = np.arange(0, len(Subj))
print(TotalSubj)
for i in TotalSubj:
    path = pathSTART + Subj[i] + "/"
    print("SUBJECT FROM:")
    print(path)
    RunAnalysis(path)
    print()
    print()
    print()



# In[ ]:


/Users/Fiammetta/Desktop/tirocinio/SuperRecognizers/TryConsistencyAnalysis/Data/
['S8']
[0]
SUBJECT FROM:
/Users/Fiammetta/Desktop/tirocinio/SuperRecognizers/TryConsistencyAnalysis/Data/S8/
['Bootstrap_Within-subject_consistency.png', 'Bootstrap_Within-subject_consistency_3bins.png', 'S8_AdjustEyeTrack_2019_Dec_04_1101 copy.csv']
['S8_AdjustEyeTrack_2019_Dec_04_1101 copy.csv']
FILES FOUND AND READ: ['S8_AdjustEyeTrack_2019_Dec_04_1101 copy.csv']
1 file_csv(s):
 85 trials/blocks. 3 blocks - each file_csv.
 255 trials - each file_csv.
 3 total blocks TOTAL. 255 total trials TOTAL.
[[[-64.0, 18.0, 3.0], [-4.0, -4.0, -10.0, 0.0, 3.0, 0.0, 7.0], [-1.0, -8.0, -18.0, 6.0, -8.0], [-7.0, -5.0, -8.0, -1.0], [-8.0, -6.0, -4.0, 9.0, -7.0], [-4.0, -6.0, 0.0, -3.0, -4.0], [31.0, 4.0, -9.0, 4.0, -68.0], [-1.0, 4.0, 1.0, 1.0, -2.0, 23.0, -8.0], [-1.0, -1.0, 3.0, 9.0, -6.0], [51.0, -2.0, -1.0, -2.0, -2.0, -4.0], [-1.0, -1.0, 1.0, -2.0, -2.0, -10.0], [-2.0, -3.0, -4.0, -8.0, 1.0, -7.0], [-5.0, 5.0, 2.0,

In [2]:
#print(TotBins_allsubjects[9])

a= np.arange(1,147,3)
print(a, len(a))

[  1   4   7  10  13  16  19  22  25  28  31  34  37  40  43  46  49  52
  55  58  61  64  67  70  73  76  79  82  85  88  91  94  97 100 103 106
 109 112 115 118 121 124 127 130 133 136 139 142 145] 49


In [10]:
### Within-subject consistency--> Bootstrap method to estimate split-half correlations 
from lmfit.models import GaussianModel
from scipy.stats import linregress
import random
import scipy.stats as stats


def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

    
def flatten(seq): 
    l = []
    for elt in seq:
        t = type(elt)
        if t is tuple or t is list:
            for elt2 in flatten(elt):
                l.append(elt2)
        else:
            l.append(elt)
    return l




#On each iteration, for each observer and each binned morph, randomly split the responses into two halves 

All_iterations_Average_r_Pearson = []
iterations = 0


#CorrSubj = []
#for b in range (1000):
#    bin_n1 = []
#    bin_n2 = []
#    Tot_MEANbin_1 = []
#    Tot_MEANbin_2 = []
#    NAN_ = 0
#    iterations += 1
        

    
    
    
ALL_CorrSubj = []
for x in range(len(Subj)):
    CorrSubj = []
    for b in range (1000):
        mean_bin1_singleSubject = []
        mean_bin2_singleSubject = []
        for bin in range(49):
            #print(bin)
            #randomly shuffle the values in the bin
            actual_bin = (TotBins_allsubjects[x])[bin]
            random.shuffle(actual_bin)
            
            #randomly split the responses into two halves 
            a, b = split_list(actual_bin)
            
            if len(a) > len(b):
                del a[-1]
            if len(b) > len(a):
                del b[-1]
                            
            
           #alert if a bin is = []        
            if a == [] and b == []:
                a = 0
                b = 0
                #print("NANN")
                #print(bin)
           #if a bin = [], repeat the value of the other bin (this is to avoid NaN)
            if a == []:
                a = b
                #NAN_ = NAN_ + 1
                #print("NAN")
                
            if b == []:
                b = a
                #NAN_ = NAN_ + 1
                #print("NAN")
    
            #calculate the mean response errors for each half
            mean_bin1 = mean(a)
            mean_bin2 = mean(b)
            
            mean_bin1_singleSubject.append(mean_bin1)
            mean_bin2_singleSubject.append(mean_bin2)
            
        #The two halves are correlated--> Pearson’s r value  
        corr, _ = pearsonr(mean_bin1_singleSubject, mean_bin2_singleSubject)
        #print(corr)
        corr = np.arctanh(corr)
        #print(corr)
        CorrSubj.append(corr)

    print(mean(CorrSubj))
    ALL_CorrSubj.append(mean(CorrSubj))
    
print(ALL_CorrSubj)
print(mean(ALL_CorrSubj))
#z_values = np.arctanh(CorrSubj)
#z_values_mean= mean(z_values)
#print(z_values_mean)

        
        
#
#    #The two halves are correlated--> Pearson’s r value  
#    Tot_halves_correlations = []
#    Tot_z_values = []
#    for x in range(len(Tot_MEANbin_1)):
#        Pvalue_halves = linregress(Tot_MEANbin_1[x], Tot_MEANbin_2[x])
#        Tot_halves_correlations.append(Pvalue_halves[2])
#        
#        #Pearson’s r value is transformed into a Fisher z value 
#        #print(Pvalue_halves[2])
#        z_values = np.arctanh(Pvalue_halves[2])
#        #print(z_values)
#        Tot_z_values.append(z_values)
#        
#        
#    #print(Tot_halves_correlations)
#    #print(Tot_z_values)
#    
#    #average z values of all the subjects
#    Average_Tot_z_values = mean(Tot_z_values)
#    #print(Average_Tot_z_values)
#    
#    #trasform the average z value in r value 
#    Average_r_Pearson = np.tanh(Average_Tot_z_values)
#    All_iterations_Average_r_Pearson.append(Average_r_Pearson)
#    
#    print("NUMERO DI NAN: ", NAN_)
#    print("ITERATIONS: ", iterations)
    
#print(All_iterations_Average_r_Pearson)
    

    
##HISTOGRAM BOOTSTRAP DATASET
#fig5 = plt.figure(figsize=(10,8))
#
#plt.hist(All_iterations_Average_r_Pearson, bins=100)
#plt.xlabel('Bootstrap r coefficient of correlation\n',  fontsize = 15)
#plt.axis([0, 0.50, 0, 40])
#plt.yticks(range(0, 45, 5))
#plt.xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])
#
#meanAmp = np.mean(All_iterations_Average_r_Pearson)
#conf_interval1 = np.percentile(All_iterations_Average_r_Pearson,[2.5])
#conf_interval2 = np.percentile(All_iterations_Average_r_Pearson,[97.5])
#print(meanAmp)
#print(conf_interval1)
#print(conf_interval2)
#
##p-value
#TotBoot_means2 = np.array(All_iterations_Average_r_Pearson)
#s0 = 0
#pval = 1-(sum(TotBoot_means2 >= s0)/len(TotBoot_means2))
#print(pval)
#
#title = "IDENTITY SD EXPERIMENT\n" + "Mean: " + str(np.round(meanAmp, decimals = 3)) + "  CI low: " + str(np.round(conf_interval1, decimals = 3)) + "  CI high: " + str(np.round(conf_interval2, decimals = 3))+ "\np-value: " + str(round(pval, 3))
#
#plt.title(title, fontsize = 20)
#
#plt.vlines(conf_interval1, 0, 40, colors='k', linestyles='solid')
#plt.vlines(conf_interval2, 0, 40, colors='k', linestyles='solid')
#plt.vlines(meanAmp, 0, 40, colors='k', linestyles='--')
#
#
#
#plt.show()
#
#FigName = "/Users/" + FolderPathUsers + "/SuperRecognizers/TryConsistencyAnalysis/all_graphs/Bootstrap_Within-subjects_3bin.png"
##FigName = path + ('Bootstrap_' + BackOrForward + str(nBack) + '_Data.png')
#
#fig5.tight_layout()
#fig5.savefig(FigName, dpi=600)
#
#plt.clf()    
#
#

-0.04465513647774614
[-0.04465513647774614]
-0.04465513647774614


In [4]:
print(All_iterations_Average_r_Pearson)

[]


In [5]:
#PLOT BAR GRAPH WITHIN-SUBJECT CONSISTENCY

list1 = [1]

fig1 = plt.figure(figsize=(10,8))

plt.bar(list1, meanAmp, width=0.8, align='center')

plt.axis([0, 2, -0.1, 0.8]) 
#plt.xticks(["Control group"])
#plt.yticks(np.arange(0, 60, 5))

plt.errorbar(list1, meanAmp, yerr=(conf_interval1-meanAmp, meanAmp-conf_interval2), fmt=' ', ecolor='black', capsize=5)

title = "WITHIN-SUBJECT CONSISTENCY\n" + "Mean: " + str(np.round(meanAmp, decimals = 2))  
plt.title(title, fontsize = 20)
plt.xlabel("\nRadiologist group", fontsize = 20)
plt.ylabel("Pearson's R", fontsize = 20)
plt.tick_params(labelsize = 15 ,bottom=False, labelbottom=False)
plt.legend()

plt.show()

FigName = "/Users/" + FolderPathUsers + "/SuperRecognizers/TryConsistencyAnalysis/all_graphs/Bootstrap_Within-subjects_BarGraph_3bin.png"


fig1.tight_layout()
fig1.savefig(FigName, dpi=600)

plt.clf()



NameError: name 'meanAmp' is not defined

<Figure size 720x576 with 0 Axes>